In [ ]:
import sys; sys.path.append('..')
import numpy as np, elastic_rods
from bending_validation import suppress_stdout as so
from linkage_vis import LinkageViewer

l = elastic_rods.RodLinkage('data/20190814_145136_linemodel.obj', 20)
driver=l.centralJoint()

mat = elastic_rods.RodMaterial('rectangle', 20000, 0.3, [11.7, 8.1])
l.setMaterial(mat)

with so(): elastic_rods.restlen_solve(l)
jdo = l.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
# with so(): elastic_rods.compute_equilibrium(l, fixedVars=fixedVars)

view = LinkageViewer(l, width=1024)
view.show()

In [ ]:
from open_linkage import open_linkage
def equilibriumSolver(tgtAngle, l, opts, fv):
    opts.beta = 1e-8
    opts.gradTol = 1e-4
    opts.useIdentityMetric = False
    return elastic_rods.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)
with so(): open_linkage(l, driver, np.deg2rad(90) - l.averageJointAngle, 10, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=True);

In [ ]:
# Load the collection of stiffening boxes from florin's box file
rbc = elastic_rods.RectangularBoxCollection('data/20190819_170613_connection_sleeves.rbc')

In [ ]:
# Visualize the stiffening box regions
view.update(preserveExisting=False, l=rbc)
view.update(preserveExisting=True, l=l)

In [ ]:
# Multiply the bending/twisting stiffness in these regions by 5
l.stiffenRegions(rbc, 5.0)

In [7]:
stiffnesses = [stiffness.lambda_1 for s in l.segments() for stiffness in s.rod.bendingStiffnesses()]
np.min(stiffnesses), np.max(stiffnesses)

(21621775.500000004, 108108877.50000001)